In [15]:
import torch
print(torch.__version__)  #注

0.4.1


In [14]:
print(torch.cuda.is_available()) 

True


In [11]:

#coding:utf8
import codecs
import sys
import pandas as pd
import numpy as np
from collections import deque#双端队列  




In [12]:
Relation2id = {'unknown': 0, '父母': 1, '夫妻': 2, 
               '师生': 3, '兄弟姐妹': 4, '合作': 5, 
               '情侣': 6, '祖孙': 7, '好友': 8,
               '亲戚': 9, '同门': 10, '上下级': 11}
print(Relation2id)

{'unknown': 0, '父母': 1, '夫妻': 2, '师生': 3, '兄弟姐妹': 4, '合作': 5, '情侣': 6, '祖孙': 7, '好友': 8, '亲戚': 9, '同门': 10, '上下级': 11}


In [18]:
import os

         
#处理数据集
datas = deque()#记录单词
labels = deque()
positionE1 = deque()
positionE2 = deque()
pos_tag = deque()#记录所有词性
parsers = deque()#记录依存分析特征
Roles = []#记录语义角色
e1_role = []
e2_role = []
total_data=0
count = [0,0,0,0,0,0,0,0,0,0,0,0]#

#加载分词相关文件
LTP_DATA_DIR = 'E:\litao\Relation_Extraction\SelfAtt_BLSTM_CH\ltp_data_v3.4.0'  # ltp模型目录的路径
from pyltp import Segmentor
cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')  # 分词模型路径，模型名称为`cws.model`
segmentor = Segmentor()  # 初始化实例
segmentor.load(cws_model_path)  # 加载模型

#加载词性相关文件
pos_model_path = os.path.join(LTP_DATA_DIR, 'pos.model')  # 词性标注模型路径，模型名称为`pos.model`
from pyltp import Postagger
postagger = Postagger() # 初始化实例
postagger.load(pos_model_path)  # 加载模型

#加载句法分析相关文件
par_model_path = os.path.join(LTP_DATA_DIR, 'parser.model')  # 依存句法分析模型路径，模型名称为`parser.model`
from pyltp import Parser
parser = Parser() # 初始化实例
parser.load(par_model_path)  # 加载模型

#加载语义角色标注相关文件
srl_model_path = os.path.join(LTP_DATA_DIR, 'pisrl_win.model')  # 语义角色标注模型目录路径，模型目录为`srl`。注意该模型路径是一个目录，而不是一个文件。
from pyltp import SementicRoleLabeller
labeller = SementicRoleLabeller() # 初始化实例
labeller.load(srl_model_path)  # 加载模型
wt = open('E:\litao\Relation_Extraction\SelfAtt_BLSTM_CH\data\parsers.txt','w',encoding='utf-8')
wr = open('E:\litao\Relation_Extraction\SelfAtt_BLSTM_CH\data\\roles.txt','w',encoding='utf-8')
with codecs.open('E:\litao\Relation_Extraction\SelfAtt_BLSTM_CH\data\data.txt','r','utf-8') as tfc: 
    for lines in tfc:
        line = lines.split()#切分
        
        #print(line[2])
        if count[Relation2id[line[2]]] <1700:

            sentence = []
            #index:获取实体在句子中的索引
            index1 = line[3].index(line[0])
            #print(index1)
            position1 = []
            index2 = line[3].index(line[1])
            #print(index2)
            position2 = []
           
            #pyltp分词
            #print(line[3])
         
            words = segmentor.segment(line[3])  # 分词
          #  print(line[3])
            #print('|'.join(words))
           # words = ' '.join(words)
           # print(words)
            words= " ".join(words).split()
            
            """
            #去停用词
            for word in words:
              if word not in stpwrdlst:
                 #print(word)
                 clean_words.append(word)"""
            
            #pylt词性标注 
            postags = postagger.postag(words)  # 词性标注
            #print(postags)
            postags= ' '.join(postags).split()
            #print(postags)
            pos_tag.append(postags)
            
            #依存句法分析
            arcs = parser.parse(words, postags)  # 句法分析
            
            for arc in arcs:
               #arc.relation = ' '.join(arc.relation)
               #print(arc.relation)
               parsers.append(arc.relation)
               wt.write(arc.relation)
               wt.write(' ')
            wt.write('\n')
            """
            for  i in range(len(parsers)):
                 if parsers[i] == 0:
                  e1_pos.append(i - index1)
                  e2_pos.append(i - index2)
            #print(e1_pos)     
      
            """
            #print( parsers)
         
            #语义角色标注
            roles = labeller.label(words, postags, arcs)  # 语义角色标注
            for role in roles:
                for arg in role.arguments:
                   #print(arg.name)
                   wr.write(arg.name)
                   wr.write(' ')
            wr.write('\n') 
                   # Roles.append(arg.name)
             #print(Roles)       
           # e1_role.append(Roles[index1])
           # e2_role.append(Roles[index2])
            #print(Roles)
          

            for i,word in enumerate(words):
                sentence.append(word)
                #print(word,sentence)
                #print(sentence)
                position1.append(i-3-index1)
                position2.append(i-3-index2)
                i+=1
            datas.append(sentence)
           # print(line[2])
            labels.append(Relation2id[line[2]])
            #print(labels)
            positionE1.append(position1)
            positionE2.append(position2)
        count[Relation2id[line[2]]]+=1   
        total_data+=1
    
segmentor.release()  # 释放模型 
postagger.release() 
#recognizer.release() 
parser.release()  
labeller.release()  
print (len(datas),len(pos_tag),len(positionE1),len(positionE2))
#返回的是队列对象，需要通过pop()出栈操作去访问队列值或者通过循环访问队列值

#print(datas.pop())
#for tag in pos_tag:
#    print(tag)








20400 20400 20400 20400


In [19]:
relation = []
with  codecs.open('E:\litao\Relation_Extraction\SelfAtt_BLSTM_CH\data\parsers.txt','r',encoding='utf-8') as f:
  for lines in f:
     # print(lines)
      line =lines.split()
      relation.append(line) 
print(len(relation)) 

20400


In [20]:
roles = []
with  codecs.open('E:\litao\Relation_Extraction\SelfAtt_BLSTM_CH\data\\roles.txt','r',encoding='utf-8') as f:
  for lines in f:
    
      line =lines.split()
      #print(lines)
      roles.append(line) 
print(len(roles))  

20400


In [23]:
#处理语义角色特征
#通过pd.Series将parers里的词做成id-relation
role = flatten(roles)
sr_allrole = pd.Series(role)

#统计每个依存关系出现的次数
sr_allrole = sr_allrole.value_counts()
#print(sr_allpostag)
#去重
set_role = sr_allrole.index
#print(set_words)
#生成id集合
set_roleids = range(1, len(set_role)+1)
#print(set_ids[:5])
#生成postag-id表,去重
role2id = pd.Series(set_roleids, index=set_role)
#print(word2id)
#生成id-relation表
id2role = pd.Series(set_role, index=set_roleids)

#在e1_relation2id表的末尾加上"BLANK"和"UNKNOW"
role2id["BLANK"]=len(role2id)+1
role2id["UNKNOW"]=len(role2id)+1
#在e1_id2relation表的末尾加上"BLANK"和"UNKNOW"
id2role[len(id2role)+1]="BLANK"
id2role[len(id2role)+1]="UNKNOW"
print(id2role)

1         A1
2         A0
3        ADV
4        TMP
5        DIS
6        LOC
7         A2
8        MNR
9        DIR
10       PRP
11       BNF
12       EXT
13       TPC
14      C-A0
15        A3
16      C-A1
17       FRQ
18       CND
19        A4
20     BLANK
21    UNKNOW
dtype: object


id表

In [22]:
#将队列里的列表展平
import collections
def flatten(x):
    def iselement(e):
        return not(isinstance(e, collections.Iterable) and not isinstance(e, str))
    for el in x:
        if iselement(el):
            yield el
        else:
            yield from flatten(el) 

"""
def flatten(x):    
    result = [] 
    for el in x:        
        if isinstance(x, collections.Iterable) and not isinstance(el, str):            
            result.extend(flatten(el))        
        else:            
            result.append(el)    
    return result
"""

""" 
print (all_words.__next__)
for i in all_words:
    print(i)
"""
#处理依存分析特征
#通过pd.Series将parers里的词做成id-relation
sr_allrelation = pd.Series(parsers)

#统计每个依存关系出现的次数
sr_allrelation = sr_allrelation.value_counts()
#print(sr_allpostag)
#去重
set_relation = sr_allrelation.index
#print(set_words)
#生成id集合
set_relationids = range(1, len(set_relation)+1)
#print(set_ids[:5])
#生成postag-id表,去重
relation2id = pd.Series(set_relationids, index=set_relation)
#print(word2id)
#生成id-relation表
id2relation = pd.Series(set_relation, index=set_relationids)

#在e1_relation2id表的末尾加上"BLANK"和"UNKNOW"
relation2id["BLANK"]=len(relation2id)+1
relation2id["UNKNOW"]=len(relation2id)+1
#在e1_id2relation表的末尾加上"BLANK"和"UNKNOW"
id2relation[len(id2relation)+1]="BLANK"
id2relation[len(id2relation)+1]="UNKNOW"
print(id2relation)




#使用列表推导式将列表平铺成1行
all_pos_tag = [num for elem in pos_tag for num in elem]
#处理词性特征
#通过pd.Series将all_pos_tag里的词做成id-词性
sr_allpostag = pd.Series(all_pos_tag)
#print(sr_allpostag)
#统计每个词性出现的次数
sr_allpostag = sr_allpostag.value_counts()
#print(sr_allpostag)
#把词性单独提出来，形成词性集合
set_postag = sr_allpostag.index
#print(set_words)
#生成id集合
set_postagids = range(1, len(set_postag)+1)
#print(set_ids[:5])
#生成postag-id表,去重
postag2id = pd.Series(set_postagids, index=set_postag)
#print(word2id)
#生成id-postag表
id2postag = pd.Series(set_postag, index=set_postagids)
#print(id2word)
#在postag2id表的末尾加上"BLANK"和"UNKNOW"
postag2id["BLANK"]=len(postag2id)+1
postag2id["UNKNOW"]=len(postag2id)+1
#在id2postag表的末尾加上"BLANK"和"UNKNOW"
id2postag[len(id2postag)+1]="BLANK"
id2postag[len(id2postag)+1]="UNKNOW"
#print ("id2postag",id2postag)
#print (len(id2postag))
#print (postag2id)


#返回的是生成器对象,需要通过__next__()获取或者循环读出
all_words = flatten(datas)
#处理words
#通过pd.Series将all_words里的词做成id-词
sr_allwords = pd.Series(all_words)
#print(sr_allwords)
#统计每个词出现的次数
sr_allwords = sr_allwords.value_counts()
#print(sr_allwords)
#把词单独提出来，形成词集合，去重
set_words = sr_allwords.index
#print(set_words)
#生成id集合
set_ids = range(1, len(set_words)+1)
#print(set_ids[:5])
#生成word-id表
word2id = pd.Series(set_ids, index=set_words)
#print(word2id)
#生成id-word表
id2word = pd.Series(set_words, index=set_ids)
#print(id2word)
#在word2id表的末尾加上"BLANK"和"UNKNOW"
word2id["BLANK"]=len(word2id)+1
word2id["UNKNOW"]=len(word2id)+1
#在id2word表的末尾加上"BLANK"和"UNKNOW"
id2word[len(id2word)+1]="BLANK"
id2word[len(id2word)+1]="UNKNOW"
print ("id2word",id2word)

1        ATT
2         WP
3        COO
4        ADV
5        VOB
6        SBV
7        RAD
8        HED
9        POB
10       LAD
11       CMP
12       DBL
13       FOB
14       IOB
15     BLANK
16    UNKNOW
dtype: object
id2word 1             ，
2             、
3             的
4             。
5             ：
6             是
7             在
8             -
9             与
10            《
11            》
12            （
13            了
14            和
15            ）
16            为
17            “
18            ”
19            一
20            人
21            之
22            有
23            等
24            后
25            他
26            中
27            不
28            也
29            于
30            聽
          ...  
39739       上弦月
39740        汉市
39741       冯异想
39742       黑龙江
39743        后记
39744       史学科
39745        缺口
39746      甄子丹汪
39747       白宝山
39748       织毛袜
39749      代杉重矩
39750       西城区
39751        蓉帐
39752       欧莱雅
39753        温情
39754        照面
39755       温兆伦
39

In [13]:
max_len = 50#定义序列长度
#处理word，把word转成id
#不够序列长度的，进行padding
def X_padding(words):
    """把 words 转为 id 形式，并自动补全位 max_len 长度。"""
    ids = []
    #如果词在word2id表中，则返回其id,否则返回未知词"UNKNOW"的id
    for i in words:
        if i in word2id:
            ids.append(word2id[i])
        else:
            ids.append(word2id["UNKNOW"])
    #如果ids的长度大于等于最大序列长度则将超出的部分截取掉，否则将ids进行扩展 
    if len(ids) >= max_len: 
        return ids[:max_len]
    ids.extend([word2id["BLANK"]]*(max_len-len(ids))) 

    return ids
def p_padding(pos):
    """把 词性 转为 id 形式，并自动补全位 max_len 长度。"""
    ids = []
    #如果词在word2id表中，则返回其id,否则返回未知词"UNKNOW"的id
    for i in pos:
        if i in postag2id:
            ids.append(postag2id[i])
        else:
            ids.append(postag2id["UNKNOW"])
    #如果ids的长度大于等于最大序列长度则将超出的部分截取掉，否则将ids进行扩展 
    if len(ids) >= max_len: 
        return ids[:max_len]
    ids.extend([postag2id["BLANK"]]*(max_len-len(ids))) 

    return ids

def R_padding(relation):
    """把 依存特征 转为 id 形式，并自动补全位 max_len 长度。"""
    ids = []
    #如果词在word2id表中，则返回其id,否则返回未知词"UNKNOW"的id
    for i in relation:
        if i in relation2id:
            ids.append(relation2id[i])
        else:
            ids.append(relation2id["UNKNOW"])
    #如果ids的长度大于等于最大序列长度则将超出的部分截取掉，否则将ids进行扩展 
    if len(ids) >= max_len: 
        return ids[:max_len]
    ids.extend([relation2id["BLANK"]]*(max_len-len(ids))) 

    return ids
def Role_padding(role):
    """把 依存特征 转为 id 形式，并自动补全位 max_len 长度。"""
    ids = []
    #如果词在word2id表中，则返回其id,否则返回未知词"UNKNOW"的id
    for i in role:
        if i in role2id:
            ids.append(role2id[i])
        else:
            ids.append(role2id["UNKNOW"])
    #如果ids的长度大于等于最大序列长度则将超出的部分截取掉，否则将ids进行扩展 
    if len(ids) >= max_len: 
        return ids[:max_len]
    ids.extend([role2id["BLANK"]]*(max_len-len(ids))) 

    return ids
 
#处理位置特征
def pos(num):
    if num<-40:
        return 0
    if num>=-40 and num<=40:
        return num+40
    if num>40:
        return 80
     
#位置特征padding
def position_padding(words):
    words = [pos(i) for i in words]
    if len(words) >= max_len:  
        return words[:max_len]
    words.extend([81]*(max_len-len(words))) 
    return words



In [1]:


#用pandas处理数据
df_data = pd.DataFrame({'words': datas, 'tags': labels,'positionE1':positionE1,'positionE2':positionE2,'pos':pos_tag,'relation':relation,'role':roles}, index=range(len(datas)))
#word和pos这两列的值应用自己定义的X_padding函数
df_data['words'] = df_data['words'].apply(X_padding)
df_data['pos'] = df_data['pos'].apply(p_padding)
df_data['relation'] = df_data['relation'].apply(R_padding)
df_data['role'] = df_data['role'].apply(Role_padding)
print(df_data['words'])
print(df_data['pos'])
print(df_data['relation'])
print(df_data['role'])
df_data['tags'] = df_data['tags']
df_data['positionE1'] = df_data['positionE1'].apply(position_padding)
df_data['positionE2'] = df_data['positionE2'].apply(position_padding)

datas = np.asarray(list(df_data['words'].values))#单词
labels = np.asarray(list(df_data['tags'].values))#类别标签
positionE1 = np.asarray(list(df_data['positionE1'].values))#位置1
positionE2 = np.asarray(list(df_data['positionE2'].values))#位置2
pos = np.asarray(list(df_data['pos'].values))#单词词性
relation = np.asarray(list(df_data['relation'].values))
role = np.asarray(list(df_data['role'].values))
print(relation.shape)
print(datas.shape)
print(labels.shape)
print(positionE1.shape)
print(positionE2.shape)
print(pos.shape)
print(role.shape)

#开始保存
import pickle
with open('E:\litao\Relation_Extraction\SelfAtt_BLSTM_CH\data\people_relation_train.pkl', 'wb') as outp:
	pickle.dump(word2id, outp)
	pickle.dump(id2word, outp)
	pickle.dump(postag2id, outp)
	pickle.dump(Relation2id, outp) 
	pickle.dump(datas, outp)
	pickle.dump(labels, outp)
	pickle.dump(positionE1, outp)
	pickle.dump(positionE2, outp)
	pickle.dump(pos, outp)
	pickle.dump(relation, outp)
	pickle.dump(role, outp)  
print ('** Finished saving the train data.')




  
  

  



NameError: name 'pd' is not defined



处理测试集

In [19]:
import os
#处理和保存测试数据

datas = deque()
labels = deque()
positionE1 = deque()
positionE2 = deque()
pos_tag =deque()#记录所有词性

e1_role_t = []
e2_role_t = []
count = [0,0,0,0,0,0,0,0,0,0,0,0]
"""
clean_words = []#接收除去停用词的单词
#从文件导入停用词表
stpwrdpath = "drive/stopwords_sum.txt"
stpwrd_dic = open(stpwrdpath, 'rb')
stpwrd_content = stpwrd_dic.read()
#将停用词表转换为list  
stpwrdlst = stpwrd_content.splitlines()
print(len(stpwrdlst))
stpwrd_dic.close()
"""
#加载分词相关文件
LTP_DATA_DIR = 'E:\litao\Relation_Extraction\SelfAtt_BLSTM_CH\ltp_data_v3.4.0'  # ltp模型目录的路径
from pyltp import Segmentor
cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')  # 分词模型路径，模型名称为`cws.model`
segmentor = Segmentor()  # 初始化实例
segmentor.load(cws_model_path)  # 加载模型

#加载词性相关文件
pos_model_path = os.path.join(LTP_DATA_DIR, 'pos.model')  # 词性标注模型路径，模型名称为`pos.model`
from pyltp import Postagger
postagger = Postagger() # 初始化实例
postagger.load(pos_model_path)  # 加载模型

#加载句法分析相关文件
par_model_path = os.path.join(LTP_DATA_DIR, 'parser.model')  # 依存句法分析模型路径，模型名称为`parser.model`
from pyltp import Parser
parser = Parser() # 初始化实例
parser.load(par_model_path)  # 加载模型

#加载语义角色标注相关文件
srl_model_path = os.path.join(LTP_DATA_DIR, 'pisrl_win.model')  # 语义角色标注模型目录路径，模型目录为`srl`。注意该模型路径是一个目录，而不是一个文件。
from pyltp import SementicRoleLabeller
labeller = SementicRoleLabeller() # 初始化实例
labeller.load(srl_model_path)  # 加载模型
wt = open('E:\litao\Relation_Extraction\SelfAtt_BLSTM_CH\data\parsers_t.txt','w',encoding='utf-8')
wr = open('E:\litao\Relation_Extraction\SelfAtt_BLSTM_CH\data\\roles_t.txt','w',encoding='utf-8')
with codecs.open('E:\litao\Relation_Extraction\SelfAtt_BLSTM_CH\data\data.txt','r','utf-8') as tfc: 
    for lines in tfc:
        line = lines.split()#切分
        #关系类别数大于1500并且小于等于1800的，作为测试样本
        if count[Relation2id[line[2]]] >1500 and count[Relation2id[line[2]]]<=1800:
        #if count[relation2id[line[2]]] <=1500:
            sentence = []
            index1 = line[3].index(line[0])
            position1 = []
            index2 = line[3].index(line[1])
            position2 = []
            #pyltp分词
            words = segmentor.segment(line[3])  # 分词
            words= " ".join(words).split()
            """
            #去停用词
            for word in words:
              if word not in stpwrdlst:
                 #print(word)
                 clean_words.append(word)"""
            
            #pylt词性标注 
            postags = postagger.postag(words)  # 词性标注
            postags= ' '.join(postags).split()
            #print(postags)
            pos_tag.append(postags)
            
            #依存句法分析
            arcs = parser.parse(words, postags)  # 句法分析
            for arc in arcs:
               #arc.relation = ' '.join(arc.relation)
               wt.write(arc.relation)
               wt.write(' ')
           
            wt.write('\n' ) 
            """
            for  i in range(len(parsers)):
                 if parsers[i] == 0:
                  e1_pos.append(i - index1)
                  e2_pos.append(i - index2)
            #print(e1_pos)     
      
            """
            #print( parsers)
            
            #语义角色标注
            roles = labeller.label(words, postags, arcs)  # 语义角色标注
            for role in roles:
              for arg in role.arguments:
                  #Roles.append(arg.name)
                  wr.write(arg.name)
                  wr.write(' ')
           
            wr.write('\n' ) 
           # print(Roles)       
          #  e1_role.append(Roles[index1])
           # e2_role.append(Roles[index2])
           # print(Roles)
         
            for i,word in enumerate(words):
                sentence.append(word)
                position1.append(i-3-index1)
                position2.append(i-3-index2)
                i+=1
            datas.append(sentence)
            labels.append(Relation2id[line[2]])
            positionE1.append(position1)
            positionE2.append(position2)
        count[Relation2id[line[2]]]+=1
segmentor.release()  # 释放模型 
postagger.release()
parser.release()
print(len(datas))
       

3600


In [20]:
relation = []
with  codecs.open('E:\litao\Relation_Extraction\SelfAtt_BLSTM_CH\data\parsers_t.txt','r',encoding='utf-8') as f:
  for lines in f:
      line =lines.split()
      relation.append(line) 
print(len(relation)) 

3600


In [21]:
roles = []
with  codecs.open('E:\litao\Relation_Extraction\SelfAtt_BLSTM_CH\data\\roles_t.txt','r',encoding='utf-8') as f:
  for lines in f:
      line =lines.split()
      roles.append(line) 
print(len(roles)) 

3600


In [22]:

#用pandas处理数据
df_data = pd.DataFrame({'words': datas, 'tags': labels,'positionE1':positionE1,'positionE2':positionE2,'pos':pos_tag,'relation':relation,'role':roles}, index=range(len(datas)))
#word和pos这两列的值应用自己定义的X_padding函数
df_data['words'] = df_data['words'].apply(X_padding)
df_data['pos'] = df_data['pos'].apply(p_padding)
df_data['relation'] = df_data['relation'].apply(R_padding)
df_data['role'] = df_data['role'].apply(Role_padding)
print(df_data['words'])
print(df_data['pos'])
print(df_data['role'])
df_data['tags'] = df_data['tags']
df_data['positionE1'] = df_data['positionE1'].apply(position_padding)
df_data['positionE2'] = df_data['positionE2'].apply(position_padding)

datas = np.asarray(list(df_data['words'].values))#单词
labels = np.asarray(list(df_data['tags'].values))#类别标签
positionE1 = np.asarray(list(df_data['positionE1'].values))#位置1
positionE2 = np.asarray(list(df_data['positionE2'].values))#位置2
pos = np.asarray(list(df_data['pos'].values))#单词词性
relation = np.asarray(list(df_data['relation'].values))
role = np.asarray(list(df_data['role'].values))

#开始保存
import pickle
with open('E:\litao\Relation_Extraction\SelfAtt_BLSTM_CH\data\people_relation_test.pkl', 'wb') as outp:
	pickle.dump(datas, outp)
	pickle.dump(labels, outp)
	pickle.dump(positionE1, outp)
	pickle.dump(positionE2, outp)
	pickle.dump(pos, outp)
	pickle.dump(relation, outp)
	pickle.dump(role, outp) 
print ('** Finished saving the test data.') 


0       [15, 11683, 703, 12054, 12515, 1632, 12, 1406,...
1       [1144, 140, 9, 255, 13522, 2530, 1, 919, 381, ...
2       [26627, 22, 4957, 2, 5249, 2, 30588, 2, 32392,...
3       [9, 1007, 22, 96, 53, 14791, 5209, 9, 2500, 3,...
4       [81, 13, 69, 4236, 20, 1329, 446, 2048, 744, 4...
5       [2, 1889, 2, 956, 10, 39258, 624, 3947, 11, 20...
6       [87, 1157, 3, 33664, 3910, 764, 1, 31582, 339,...
7       [7, 17, 7174, 18, 26, 1, 2392, 354, 655, 3, 11...
8       [6646, 6272, 5638, 216, 15411, 1, 61, 659, 147...
9       [7597, 9902, 566, 21990, 5, 43, 7, 521, 26, 83...
10      [537, 3137, 1904, 3, 17, 2844, 4893, 18, 1, 43...
11      [34133, 3324, 5, 33180, 2, 25923, 2, 24643, 2,...
12      [44, 95, 42, 43, 6065, 8, 127, 124, 9, 5790, 2...
13      [688, 2, 1456, 4134, 2745, 1, 16, 4805, 326, 1...
14      [200, 60, 120, 825, 342, 43, 1233, 350, 6432, ...
15      [869, 29, 321, 7, 19, 82, 224, 162, 26, 1, 504...
16      [177, 73, 5, 6271, 3, 73, 22, 6789, 2, 8364, 2...
17      [204, 

TypeError: 'numpy.ndarray' object is not callable

加载中文预训练词向量400000*300

In [5]:


import torch
import gensim

vocab_size = len(word2id)+1  
embed_size = 300
wvmodel = gensim.models.KeyedVectors.load_word2vec_format('E:\litao\Relation_Extraction\SelfAtt_BLSTM_CH\sgns.target.word-character.char1-2.dynwin5.thr10.neg5.dim300 - 40w.txt',
                                                          binary=False,encoding='utf-8')


weight = torch.zeros(vocab_size+1, embed_size)

for i in range(len(wvmodel.index2word)):
    try:
        index = word2id[wvmodel.index2word[i]]
        #print("yes")
    except:
        continue    
    weight[index, :] = torch.from_numpy(wvmodel.get_vector(id2word[word2id[wvmodel.index2word[i]]]))
embedding_pre = weight
print(embedding_pre[:2])
 

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  

Model

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SelfAtt_BLSTM(nn.Module):
    def __init__(self,config,embedding_pre):
        super(SelfAtt_BLSTM, self).__init__()
        
        # Set hyper parameters
        self.word_emb_dim =config['word_emb_dim']
        self.word_vocab = config['word_vocab']#单词数，词表大小
        
        
        self.pos_tag_dim = config['pos_tag_dim']
        self.pos_tag_vocab = config['pos_tag_vocab']
        
        self.relation_dim = config['relation_dim']
        self.relation_vocab = config['relation_vocab']
        
        self.role_dim = config['role_dim']
        self.role_vocab = config['role_vocab']
        
        self.hidden_dim = config['hidden_dim'] #隐藏单元个数
        self.output_dim = config['output_dim']#输出维度
        
    
        self.MAX_POS = config['MAX_POS']#位置
        self.pos_emb_dim =  config['pos_emb_dim']
        
        
        
       
        
        
        #embedding layer
        self.word_emb = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_pre),freeze=False)
        
        self.pos_tag_emb = nn.Embedding(self.pos_tag_vocab,self.pos_tag_dim)
        
        self.relation_emb = nn.Embedding(self.relation_vocab,self.relation_dim)
        
        self.role_emb = nn.Embedding(self.role_vocab,self.role_dim)
        
        self.pos1_emb = nn.Embedding(self.MAX_POS*2+1, self.pos_emb_dim)
        self.pos1_emb.weight.data.uniform_(-0.0, 0.0)
       
        self.pos2_emb = nn.Embedding(self.MAX_POS*2+1, self.pos_emb_dim)
        self.pos2_emb.weight.data.uniform_(-0.0, 0.0)
        
        #BLSTM layer
        self.rnn = nn.LSTM(self.word_emb_dim + self.pos_tag_dim+relation_dim +self.role_dim+self.pos_emb_dim * 2, hidden_dim,num_layers=1, bidirectional=True, batch_first = True)
        
        #Self-Attention layer
        self.attention_hidden = nn.Linear(hidden_dim * 2, hidden_dim,bias=False)

        self.attention = nn.Linear(hidden_dim, 1, bias=False)
        
        
        #classifier
        self.classifier = nn.Linear(hidden_dim * 2, output_dim)
  
    def forward(self, sentence,pos_tag,relation,role,pos1,pos2 ,is_train = True):
        #print("sentence.shape:",sentence.shape)
        #print("pos1.shape:",pos1.shape)
        #print("pos2.shape:",pos2.shape)
        word_embeddings = self.word_emb(sentence)
        pos_tag_embeddings = self.pos_tag_emb(pos_tag)
        relation_embeddings = self.relation_emb(relation)
        role_embeddings = self.role_emb(role)
        pos1_embeddings = self.pos1_emb(pos1)
        pos2_embeddings = self.pos2_emb(pos2)
        
        
        
        # LSTM layer
        X = torch.cat((word_embeddings,pos_tag_embeddings,relation_embeddings,role_embeddings,pos1_embeddings, pos2_embeddings),-1)
        #X = word_embeddings
        #print("word_embeddings:",X[:2])
        #print("pos1_embeddings:",pos1_embeddings[:2])
        #print("pos2_embeddings:",pos2_embeddings[:2])
       # X = F.dropout(X, p=0.7)#将词嵌入层Droupout下
        for_output,hiddens = self.rnn(X)
        #print("output.shape:",hiddens.shape)
        #rev_hiddens, rev_output = self.rev_lstm(X)
        #print("word_embeddings.shape",word_embeddings.shape)
        #print("hiddens.shape:", hiddens.shape)
        #print("pos_tag_embeddings.shape:", pos_tag_embeddings.shape)
        #print("relationembeddings.shape:",relation_embeddings.shape)
        #print("pos1_embeddings.shape:",pos1_embeddings.shape)
        #print("pos2_embeddings.shape:",pos2_embeddings.shape)
        # Self Attentive layer自注意力层
        #att_input = torch.cat((for_output,pos_tag_embeddings,relation_embeddings,pos1_embeddings, pos2_embeddings),-1)#拼接
        #print("att_input.shape:",att_input.shape)
        att_input = F.dropout(for_output, p=0.7)#拼接后的层dropout
        
        att_hidden = torch.tanh(self.attention_hidden(att_input))#feed自注意力层
        att_hidden = F.dropout(att_hidden, p=0.7)
        #print("att_hidden.shape:", att_hidden.shape)
        #att_hidden = F.dropout(att_hidden, p=0.6)#对att_hiddendropout下
        att_scores = self.attention(att_hidden)#得到注意力分数
        att_scores = F.dropout(att_scores, p=0.7)
        #print("att_scores.shape:",att_scores.shape)
        #print("att_scores.shape:", att_scores.shape)
        attention_distrib = F.softmax(att_scores, dim = 1)#对分数进行归一化,a=softmax(ws2*tanh(ws1*H.transpose))
       # print("attention_distrib.shape:",attention_distrib.shape)
        #hiddens = F.dropout(hiddens, p=0.5)#hiddens dropout下
        context_vector = torch.sum(for_output * attention_distrib, dim = 1)#得到背景向量context_vector,加权求和
        #print("context_vector.shape:",context_vector.shape)
        # Classifier
        #context_hidden = self.classifier_hidden(context_vector)
        #context_hidden = F.dropout(context_hidden, p=0.5, training=is_train)
        finals = self.classifier(context_vector)
        #finals = F.softmax(self.classifier(context_vector), dim = 1)#将背景向量送入分类器进行分类
       # print("finals.shape",finals.shape)
        return finals

load data

In [10]:
#coding:utf8
import numpy as np
import pickle
import sys
import torch
import torch.utils.data as D

#加载保存好的训练dataset.pkl
with open('E:\litao\Relation_Extraction\SelfAtt_BLSTM_CH\data\people_relation_train.pkl', 'rb') as inp:
    word2id = pickle.load(inp)
    print("word2id.shape:",word2id.shape)
    id2word = pickle.load(inp)
    postag2id = pickle.load(inp)
    Relation2id = pickle.load(inp)
    print("Relation2id.shape:",Relation2id)
    train = pickle.load(inp)
    
    labels = pickle.load(inp)
    print("labels.shape:",labels.shape)
    position1 = pickle.load(inp)
    print("position1.shape:",position1.shape)
    position2 = pickle.load(inp)
    print("position2.shape:",position2.shape)
    pos_tag = pickle.load(inp)
    relation = pickle.load(inp)
    role = pickle.load(inp)
    print("relation.shape",relation.shape)
#10折验证取两万个样本

train = train[:20000]  
labels = labels[:20000]
position1 =  position1[:20000]
position2 =  position2[:20000]  
postag = pos_tag[:20000]
Relation = relation[:20000]
#print(train[:2])


#加载保存好的测试dataset.pkl    
with open('E:\litao\Relation_Extraction\SelfAtt_BLSTM_CH\data\people_relation_test.pkl', 'rb') as inp:
    test = pickle.load(inp)
    labels_t = pickle.load(inp)
    position1_t = pickle.load(inp)
    position2_t = pickle.load(inp)
    pos_tag_t = pickle.load(inp)
    relation_t = pickle.load(inp)
    role_t = pickle.load(inp)
"""
test =test[:2500]
labels_t = labels_t[:2500]
position1_t = position1_t[:2500]
position2_t = position2_t[:2500]
pos_tag_t = pos_tag_t[:2500]
relation_t = relation_t[:2500]
"""
print ("train len", len(train))#训练样本数     
print ("test len", len(test)) #测试样本数  
print ("word2id len",len(word2id))
print("test len",len(test))


ImportError: cannot import name 'container_abcs'

In [64]:
print(torch.cuda.device_count())

1


Training

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torch.utils.data as D
from torch.autograd import Variable
import random
import numpy as np
        

#BiLSTM_SelfATT模型训练
#BiLSTM_SelfATT参数设置
word_vocab=len(word2id)+1   
word_emb_dim=300



MAX_POS = 82#不同数据集这里可能会报错。这里的pos_size一定要比数据集里的序列长度大，不然会报错
pos_emb_dim=30

pos_tag_dim = 30
pos_tag_vocab = len(postag2id)+1

relation_dim = 30
relation_vocab = 82

role_dim = 30
role_vocab = 82

hidden_dim=200

output_dim = len(Relation2id)#关系类别
print(output_dim)

BATCH = 6
EPOCHS = 50



config={}
config['word_vocab'] = word_vocab
config['word_emb_dim'] = word_emb_dim

config['pos_tag_dim'] = pos_tag_dim
config['pos_tag_vocab'] = pos_tag_vocab

config['relation_dim'] = relation_dim
config['relation_vocab'] = relation_vocab

config['role_dim'] = role_dim
config['role_vocab'] = role_vocab

config['MAX_POS'] = MAX_POS
config['pos_emb_dim'] = pos_emb_dim

config['hidden_dim'] = hidden_dim
config['output_dim'] = output_dim



learning_rate = 0.0005

 #0.0005
device= torch.device('cuda:0')

model = SelfAtt_BLSTM(config,embedding_pre)
print(model)
params = list(model.parameters())
#model.to(device)
model = model.to(device)
#model = torch.load('model/model_epoch20.pkl')

#优化器
#optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-8)
#optimizer = optim.Adamax(model.parameters(), lr=learning_rate,weight_decay=5e-4 )
optimizer = optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-8 )
torch.nn.utils.clip_grad_norm_(params, 10)

#损失函数
#criterion = nn.CrossEntropyLoss(size_average=True)
criterion = nn.CrossEntropyLoss()

#将数据转成LongTensor格式
train = torch.LongTensor(train[:len(train)-len(train)%BATCH])
print("train.shape:",train.shape)
#pos_tag = torch.LongTensor(pos_tag)
pos_tag = torch.LongTensor(pos_tag[:len(train)-len(train)%BATCH])
relation = torch.LongTensor(relation[:len(train)-len(train)%BATCH])
role = torch.LongTensor(role[:len(train)-len(train)%BATCH])
#position1 = torch.LongTensor(position1)
position1 = torch.LongTensor(position1[:len(train)-len(train)%BATCH])
position2 = torch.LongTensor(position2[:len(train)-len(train)%BATCH])
labels = torch.LongTensor(labels[:len(train)-len(train)%BATCH])

train_datasets = D.TensorDataset(train,pos_tag,relation,role,position1,position2,labels)
train_dataloader = D.DataLoader(train_datasets,BATCH,True,num_workers=2)
print("train_datasets:",train_datasets[0][0].shape)


#同上
test = torch.LongTensor(test[:len(test)-len(test)%BATCH])
pos_tag_t = torch.LongTensor(pos_tag_t[:len(test)-len(test)%BATCH])
position1_t = torch.LongTensor(position1_t[:len(test)-len(test)%BATCH])
relation_t =torch.LongTensor(relation_t[:len(test)-len(test)%BATCH])
role_t =torch.LongTensor(role_t[:len(test)-len(test)%BATCH])
#position1_t = torch.LongTensor(position1_t)
position2_t = torch.LongTensor(position2_t[:len(test)-len(test)%BATCH])
labels_t = torch.LongTensor(labels_t[:len(test)-len(test)%BATCH])

test_datasets = D.TensorDataset(test,pos_tag_t,relation_t,role_t,position1_t,position2_t,labels_t)
test_dataloader = D.DataLoader(test_datasets,BATCH,True,num_workers=2)



#start training
for epoch in range(EPOCHS):
    print ("epoch:",epoch)
    #统计预测正确的标签数
    acc=0
    #统计预测所有标签的总数
    total=0
    total_loss=0
    
    for sentence,pos_tag,relation,role,pos1,pos2,tag in train_dataloader:#批训练
          #print("sentence:",sentence)
          #print("pos1:",pos1)
          #print("pos2:",pos2)

          sentence = Variable(sentence.cuda())
          pos_tag = Variable(pos_tag.cuda())
          relation = Variable(relation.cuda())
          role = Variable(role.cuda())
          pos1 = Variable(pos1.cuda())
          pos2 = Variable(pos2.cuda())
          tags = Variable(tag.cuda())
          """
          sentence = Variable(sentence)
          pos_tag = Variable(pos_tag)
          e1_head = Variable(relation)
          pos1 = Variable(pos1)
          pos2 = Variable(pos2)
          tags = Variable(tag)
          """
          y = model(sentence,pos_tag,relation,role,pos1,pos2)
         
     
          #epoch_loss += loss.data.cpu().numpy()[0]
          loss = criterion(y, tags)
          total_loss += loss
          #loss += l2_loss(params[1:]) * l2_rate
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()    
          y = np.argmax(y.data.cpu().numpy(),axis=1)
         # print("predict:",y)
         # print("true：",tags)   
          #y = np.argmax(y.data.numpy(),axis=1)
          #print("train_y:",y)
          #print("train_tag",tag)
          for y1,y2 in zip(y,tag):
             # print("train_y1:",y1)
             # print("train_y2:",y2)
              if y1==y2:
                  acc+=1

              total+=1
      
    print ("train:",100*float(acc)/total,"%")
    print("train_loss:",(total_loss/total).data.cpu().numpy())
     # print((total_loss/len(train_dataloader.dataset)).cpu().data.numpy())
      #print((total_loss/len(train_dataloader.dataset)).data.numpy())
      #print(loss.cpu().data.numpy()) 

    
    
    #start testing
    acc_t=0
    total_t=0
    total_loss_t=0
    count_predict = [0,0,0,0,0,0,0,0,0,0,0,0,0]
    count_total = [0,0,0,0,0,0,0,0,0,0,0,0,0]
    count_right = [0,0,0,0,0,0,0,0,0,0,0,0]
    for sentence,pos_tag,relation,role,pos1,pos2,tag in test_dataloader:  

          sentence = Variable(sentence.cuda()) 
          pos_tag = Variable(pos_tag.cuda()) 
          relation = Variable(relation.cuda())
          role = Variable(role.cuda())
          pos1 = Variable(pos1.cuda())
          pos2 = Variable(pos2.cuda())
          tags = Variable(tag.cuda())
          """
          sentence = Variable(sentence)
          pos_tag = Variable(pos_tag)
          relation = Variable(relation)
          pos1 = Variable(pos1)
          pos2 = Variable(pos2)
          tag = Variable(tag)
           """
          y = model(sentence,pos_tag,relation,relation,role,pos1,pos2)
          loss = criterion(y, tags)
          total_loss_t += loss

          y = np.argmax(y.data.cpu().numpy(),axis=1)
         # y = np.argmax(y.data.numpy(),axis=1)
         # print('testy:',y)
         # print('testtag:',tag)
          for y1,y2 in zip(y,tag):
             # print("test_y1:",y1)
              #print("test_y2:",y2)
              count_predict[y1]+=1
              count_total[y2]+=1
              if y1==y2:
                  count_right[y1]+=1
              total_t+=1

    print("test_loss:",(total_loss_t/total_t).data.cpu().numpy())
    precision = [0,0,0,0,0,0,0,0,0,0,0,0,0]
    recall = [0,0,0,0,0,0,0,0,0,0,0,0,0]
    for i in range(len(count_predict)):
          if count_predict[i]!=0 :
              precision[i] = float(count_right[i])/count_predict[i]

          if count_total[i]!=0:
              recall[i] = float(count_right[i])/count_total[i]    

    precision = sum(precision)/len(Relation2id)
    recall = sum(recall)/len(Relation2id)
    print ("准确率：",precision)
    print ("召回率：",recall)
    print ("f：", (2*precision*recall)/(precision+recall))

#start saveing
torch.save(model, "E:\litao\Relation_Extraction\SelfAtt_BLSTM_CH\model\model_BiLSTM_selfAtt_ch.pkl")
print ("model has been saved")

12
SelfAtt_BLSTM(
  (word_emb): Embedding(39770, 300)
  (pos_tag_emb): Embedding(30, 30)
  (relation_emb): Embedding(82, 30)
  (role_emb): Embedding(82, 30)
  (pos1_emb): Embedding(165, 30)
  (pos2_emb): Embedding(165, 30)
  (rnn): LSTM(450, 200, batch_first=True, bidirectional=True)
  (attention_hidden): Linear(in_features=400, out_features=200, bias=False)
  (attention): Linear(in_features=200, out_features=1, bias=False)
  (classifier): Linear(in_features=400, out_features=12, bias=True)
)
**************************
self._name:  cudart64_80
**************************
**************************
self._name:  cudnn64_7
**************************
train.shape: torch.Size([19998, 50])
train_datasets: torch.Size([50])
epoch: 0
train: 55.590559055905594 %
train_loss: 0.2250144


In [27]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

# 可视化模型结构

In [ ]:
def make_dot(var, params=None):
    """ Produces Graphviz representation of PyTorch autograd graph
    Blue nodes are the Variables that require grad, orange are Tensors
    saved for backward in torch.autograd.Function
    Args:
        var: output Variable
        params: dict of (name, Variable) to add names to node that
            require grad (TODO: make optional)
    """
    if params is not None:
        assert isinstance(params.values()[0], Variable)
        param_map = {id(v): k for k, v in params.items()}

    node_attr = dict(style='filled',
                     shape='box',
                     align='left',
                     fontsize='12',
                     ranksep='0.1',
                     height='0.2')
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()

    def size_to_str(size):
        return '(' + (', ').join(['%d' % v for v in size]) + ')'

    def add_nodes(var):
        if var not in seen:
            if torch.is_tensor(var):
                dot.node(str(id(var)), size_to_str(var.size()), fillcolor='orange')
            elif hasattr(var, 'variable'):
                u = var.variable
                name = param_map[id(u)] if params is not None else ''
                node_name = '%s\n %s' % (name, size_to_str(u.size()))
                dot.node(str(id(var)), node_name, fillcolor='lightblue')
            else:
                dot.node(str(id(var)), str(type(var).__name__))
            seen.add(var)
            if hasattr(var, 'next_functions'):
                for u in var.next_functions:
                    if u[0] is not None:
                        dot.edge(str(id(u[0])), str(id(var)))
                        add_nodes(u[0])
            if hasattr(var, 'saved_tensors'):
                for t in var.saved_tensors:
                    dot.edge(str(id(t)), str(id(var)))
                    add_nodes(t)

    add_nodes(var.grad_fn)
    return dot
if __name__ == '__main__':
    net = model
    X = torch.cat((train,pos_tag,relation,role,position1,position2,),-1) 
    y = net(X)
    g = make_dot(y)
    g.view()


In [69]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [0]:
#使用grid search 来调参

import torch
import torch.nn as nn
import torch.optim as optim

import torch.utils.data as D
from torch.autograd import Variable
import random
import numpy as np
        

#BiLSTM_SelfATT模型训练
#BiLSTM_SelfATT参数设置
word_vocab=len(word2id)+1   
word_emb_dim=300



MAX_POS = 82#不同数据集这里可能会报错。这里的pos_size一定要比数据集里的序列长度大，不然会报错
pos_emb_dim=25

pos_tag_dim = 30
pos_tag_vocab = len(postag2id)+1

relation_dim = 30
relation_vocab = 82


hidden_dim=150

output_dim = len(Relation2id)#关系类别
print(output_dim)

BATCH = 16
EPOCHS = 100

l2_rate = 0.3

config={}
config['word_vocab'] = word_vocab
config['word_emb_dim'] = word_emb_dim

config['pos_tag_dim'] = pos_tag_dim
config['pos_tag_vocab'] = pos_tag_vocab

config['relation_dim'] = relation_dim
config['relation_vocab'] = relation_vocab

config['MAX_POS'] = MAX_POS
config['pos_emb_dim'] = pos_emb_dim

config['hidden_dim'] = hidden_dim
config['output_dim'] = output_dim


#将数据转成LongTensor格式
train = torch.LongTensor(train[:len(train)-len(train)%BATCH])
print("train.shape:",train.shape)
#pos_tag = torch.LongTensor(pos_tag)
pos_tag = torch.LongTensor(pos_tag[:len(train)-len(train)%BATCH])
relation = torch.LongTensor(relation[:len(train)-len(train)%BATCH])
#position1 = torch.LongTensor(position1)
position1 = torch.LongTensor(position1[:len(train)-len(train)%BATCH])
position2 = torch.LongTensor(position2[:len(train)-len(train)%BATCH])
labels = torch.LongTensor(labels[:len(train)-len(train)%BATCH])

train_datasets = D.TensorDataset(train,pos_tag,relation,position1,position2,labels)
train_dataloader = D.DataLoader(train_datasets,BATCH,True,num_workers=2)
print("train_datasets:",train_datasets[0][0].shape)


#同上
test = torch.LongTensor(test[:len(test)-len(test)%BATCH])
pos_tag_t = torch.LongTensor(pos_tag_t[:len(test)-len(test)%BATCH])
position1_t = torch.LongTensor(position1_t[:len(test)-len(test)%BATCH])
relation_t =torch.LongTensor(relation_t[:len(test)-len(test)%BATCH])
#position1_t = torch.LongTensor(position1_t)
position2_t = torch.LongTensor(position2_t[:len(test)-len(test)%BATCH])
labels_t = torch.LongTensor(labels_t[:len(test)-len(test)%BATCH])

test_datasets = D.TensorDataset(test,pos_tag_t,relation_t,position1_t,position2_t,labels_t)
test_dataloader = D.DataLoader(test_datasets,BATCH,True,num_workers=2)


f = 0.83
#learning_rate =3e-4

# 0.0005
device= torch.device('cuda:0')

model = Baseline_Model(config,embedding_pre)
print(model)
params = list(model.parameters())
#model.to(device)
model = model.cuda()
#model = torch.load('model/model_epoch20.pkl')

#损失函数
#criterion = nn.CrossEntropyLoss(size_average=True)
criterion = nn.CrossEntropyLoss()
#优化器
#optimizer = optim.SGD(model.parameters(), lr=learning_rate,momentum = 0.9, weight_decay=5e-4)
#optimizer = optim.Adamax(model.parameters(), lr=learning_rate,weight_decay=5e-4 )
#start training
for learning_rate in [0.1,0.01,0.5,0.001,0.005,0.0005]:
    for w in  [1e-8 ,5e-4,0.0001,0.0006,0.01,1]:
        for c in [1,2,3,4,5,6,9,10]:
          optimizer = optim.Adam(params, lr=learning_rate,weight_decay=w)
          torch.nn.utils.clip_grad_norm_(params, c)
         



         
          for epoch in range(EPOCHS):
              print ("epoch:",epoch)
              #统计预测正确的标签数
              acc=0
              #统计预测所有标签的总数
              total=0
              total_loss=0

              for sentence,pos_tag,relation,pos1,pos2,tag in train_dataloader:#批训练
                    #print("sentence:",sentence)
                    #print("pos1:",pos1)
                    #print("pos2:",pos2)

                    sentence = Variable(sentence.cuda())
                    pos_tag = Variable(pos_tag.cuda())
                    relation = Variable(relation.cuda())
                    pos1 = Variable(pos1.cuda())
                    pos2 = Variable(pos2.cuda())
                    tags = Variable(tag.cuda())
                    """
                    sentence = Variable(sentence)
                    pos_tag = Variable(pos_tag)
                    e1_head = Variable(relation)
                    pos1 = Variable(pos1)
                    pos2 = Variable(pos2)
                    tags = Variable(tag)
                    """
                    y = model(sentence,pos_tag,relation,pos1,pos2)  
                    #epoch_loss += loss.data.cpu().numpy()[0]
                    loss = criterion(y, tags)
                    total_loss += loss 
                    #loss += l2_loss(params[1:]) * l2_rate
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()    
                    y = np.argmax(y.data.cpu().numpy(),axis=1)
                    #y = np.argmax(y.data.numpy(),axis=1)
                    #print("train_y:",y)
                    #print("train_tag",tag)
                    for y1,y2 in zip(y,tag):
                       # print("train_y1:",y1)
                       # print("train_y2:",y2)
                        if y1==y2:
                            acc+=1

                        total+=1

              print ("train:",100*float(acc)/total,"%")
              print("train_loss:",(total_loss/total).data.cpu().numpy())
               # print((total_loss/len(train_dataloader.dataset)).cpu().data.numpy())
                #print((total_loss/len(train_dataloader.dataset)).data.numpy())
                #print(loss.cpu().data.numpy()) 



              #start testing
              acc_t=0
              total_t=0
              total_loss_t=0
              count_predict = [0,0,0,0,0,0,0,0,0,0,0,0,0]
              count_total = [0,0,0,0,0,0,0,0,0,0,0,0,0]
              count_right = [0,0,0,0,0,0,0,0,0,0,0,0]
              for sentence,pos_tag,relation,pos1,pos2,tag in test_dataloader:  

                    sentence = Variable(sentence.cuda()) 
                    pos_tag = Variable(pos_tag.cuda()) 
                    relation = Variable(relation.cuda())
                    pos1 = Variable(pos1.cuda())
                    pos2 = Variable(pos2.cuda())
                    tags = Variable(tag.cuda())
                    """
                    sentence = Variable(sentence)
                    pos_tag = Variable(pos_tag)
                    relation = Variable(relation)
                    pos1 = Variable(pos1)
                    pos2 = Variable(pos2)
                    tag = Variable(tag)
                     """
                    y = model(sentence,pos_tag,relation,relation,pos1,pos2)
                    loss = criterion(y, tags)
                    total_loss_t += loss 

                    y = np.argmax(y.data.cpu().numpy(),axis=1)
                   # y = np.argmax(y.data.numpy(),axis=1)
                   # print('testy:',y)
                   # print('testtag:',tag)
                    for y1,y2 in zip(y,tag):
                       # print("test_y1:",y1)
                        #print("test_y2:",y2)
                        count_predict[y1]+=1
                        count_total[y2]+=1
                        if y1==y2:
                            count_right[y1]+=1
                        total_t+=1

              print("test_loss:",(total_loss_t/total_t).data.cpu().numpy())
              precision = [0,0,0,0,0,0,0,0,0,0,0,0,0]
              recall = [0,0,0,0,0,0,0,0,0,0,0,0,0]
              for i in range(len(count_predict)):
                    if count_predict[i]!=0 :
                        precision[i] = float(count_right[i])/count_predict[i]

                    if count_total[i]!=0:
                        recall[i] = float(count_right[i])/count_total[i]    

              precision = sum(precision)/len(Relation2id)
              recall = sum(recall)/len(Relation2id)
              F = (2*precision*recall)/(precision+recall)
              print ("准确率：",precision)
              print ("召回率：",recall)
              print("F1：", F)
              if F>f:
                 print('lr:',learning_rate)
                 print('weight_decay:',w)
                 print('clip_grad:',c)
                 
              
#start saveing
torch.save(model, "drive/论文/ChineseNRE-master更新后/data/model_BiLSTM_selfAtt_ch.pkl")
print ("model has been saved")

training model

In [0]:
#10折交叉验证
import torch
import torch.nn as nn
import torch.optim as optim

import torch.utils.data as D
from torch.autograd import Variable
import random
import numpy as np
        

#BiLSTM_SelfATT模型训练
#BiLSTM_SelfATT参数设置
word_vocab=len(word2id)+1   
word_emb_dim=300

MAX_POS = 82#不同数据集这里可能会报错。这里的pos_size一定要比数据集里的序列长度大，不然会报错
pos_emb_dim=25

pos_tag_dim = 30
pos_tag_vocab = len(postag2id)+1

relation_dim = 30#依存关系
relation_vocab = 82


hidden_dim=200

output_dim = len(Relation2id)#关系类别
print(output_dim)

BATCH = 16
EPOCHS = 10

l2_rate = 0.3

config={}
config['word_vocab'] = word_vocab
config['word_emb_dim'] = word_emb_dim

config['pos_tag_dim'] = pos_tag_dim
config['pos_tag_vocab'] = pos_tag_vocab

config['relation_dim'] = relation_dim
config['relation_vocab'] = relation_vocab

config['MAX_POS'] = MAX_POS
config['pos_emb_dim'] = pos_emb_dim

config['hidden_dim'] = hidden_dim
config['output_dim'] = output_dim



learning_rate = 0.0005


device= torch.device('cuda:0')

model = Baseline_Model(config,embedding_pre)
print(model)
params = list(model.parameters())
#model.to(device)
model = model.cuda()
#model = torch.load('model/model_epoch20.pkl')

#优化器
#optimizer = optim.SGD(model.parameters(), lr=learning_rate,momentum = 0.9, weight_decay=5e-4)
#optimizer = optim.Adamax(model.parameters(), lr=learning_rate,weight_decay=5e-4 )
optimizer = optim.Adam(model.parameters(), lr=learning_rate,weight_decay=5e-4 )
#torch.nn.utils.clip_grad_norm_(params, 5.0)

#损失函数
#criterion = nn.CrossEntropyLoss(size_average=True)
criterion = nn.CrossEntropyLoss()
"""
#将数据转成LongTensor格式
train = torch.LongTensor(train[:len(train)-len(train)%BATCH])
print("train.shape:",train.shape)
#pos_tag = torch.LongTensor(pos_tag)
pos_tag = torch.LongTensor(pos_tag[:len(train)-len(train)%BATCH])
relation = torch.LongTensor(relation[:len(train)-len(train)%BATCH])
#position1 = torch.LongTensor(position1)
position1 = torch.LongTensor(position1[:len(train)-len(train)%BATCH])
position2 = torch.LongTensor(position2[:len(train)-len(train)%BATCH])
labels = torch.LongTensor(labels[:len(train)-len(train)%BATCH])

train_datasets = D.TensorDataset(train,pos_tag,relation,position1,position2,labels)
train_dataloader = D.DataLoader(train_datasets,BATCH,True,num_workers=2)
print("train_datasets:",train_datasets[0][0].shape)


#同上
test = torch.LongTensor(test[:len(test)-len(test)%BATCH])
pos_tag_t = torch.LongTensor(pos_tag_t[:len(test)-len(test)%BATCH])
position1_t = torch.LongTensor(position1_t[:len(test)-len(test)%BATCH])
relation_t =torch.LongTensor(relation_t[:len(test)-len(test)%BATCH])
#position1_t = torch.LongTensor(position1_t)
position2_t = torch.LongTensor(position2_t[:len(test)-len(test)%BATCH])
labels_t = torch.LongTensor(labels_t[:len(test)-len(test)%BATCH])

test_datasets = D.TensorDataset(test,pos_tag_t,relation_t,position1_t,position2_t,labels_t)
test_dataloader = D.DataLoader(test_datasets,BATCH,True,num_workers=2)

"""

#start training
for epoch in range(EPOCHS):
    print ("epoch:",epoch)
    #统计预测正确的标签数
    acc=0
    #统计预测所有标签的总数
    total=0
    total_loss=0
    e = 0
    train_p = []
    test_p = []
    test_r = []
    test_f = []
    step = 2000
    #对齐打乱数据 
    cc_pos = list(zip(train,postag,Relation,position1 ,position2,labels))
    random.shuffle(cc_pos)

    train,postag,Relation, position1,position2, labels = zip(*cc_pos)
    train = np.asarray(train)
    postag = np.asarray(postag)
    Relation = np.asarray(Relation)
    position1 = np.asarray(position1)
    position2 = np.asarray(position2)
    labels = np.asarray(labels)
    for i in range(10):
      #print(train[:e].shape)
      #print(train[e+step:])
      train_9 = np.concatenate((train[:e],train[e+step:]),0) 
      #print("train_9.shape:",train_9.shape)
      pos_tag_9 = np.concatenate((postag[:e],postag[e+step:]),0)
      #print("pos_tag_9.shape:",pos_tag_9.shape)
      relation_9 = np.concatenate((Relation[:e],Relation[e+step:]),0)
      position1_9 = np.concatenate((position1[:e],position1[e+step:]),0)
      position2_9 = np.concatenate((position2[:e],position2[e+step:]),0)
      labels_9 = np.concatenate((labels[:e],labels[e+step:]),0)
      
     
      test = train[e:e+step]
      #print(len(test))
      pos_tag_t = postag[e:e+step]
      relation_t = Relation[e:e+step]
      position1_t = position1[e:e+step]
      position2_t = position2[e:e+step]
      labels_t = labels[e:e+step]
      #s += step
      e += step
      #将数据转成LongTensor格式
      
      train_9 = torch.LongTensor(train_9[:len(train_9)-len(train_9)%BATCH])
      #print("train_9.shape:",train_9.shape)
      pos_tag_9 = torch.LongTensor(pos_tag_9[:len(train_9)-len(train_9)%BATCH])
      relation_9 = torch.LongTensor(relation_9[:len(train_9)-len(train_9)%BATCH])
      position1_9 = torch.LongTensor(position1_9[:len(train_9)-len(train_9)%BATCH])
      position2_9 = torch.LongTensor(position2_9[:len(train)-len(train)%BATCH])
      labels_9 = torch.LongTensor(labels[:len(train_9)-len(train_9)%BATCH])
      train_datasets = D.TensorDataset(train_9,pos_tag_9,relation_9,position1_9,position2_9,labels_9)
      train_dataloader = D.DataLoader(train_datasets,BATCH,True,num_workers=2)
      
      test = torch.LongTensor(test[:len(test)-len(test)%BATCH])
      pos_tag_t = torch.LongTensor(pos_tag_t[:len(test)-len(test)%BATCH])
      relation_t =torch.LongTensor(relation_t[:len(test)-len(test)%BATCH])
      position1_t = torch.LongTensor(position1_t[:len(test)-len(test)%BATCH])
      #position1_t = torch.LongTensor(position1_t)
      position2_t = torch.LongTensor(position2_t[:len(test)-len(test)%BATCH])
      labels_t = torch.LongTensor(labels_t[:len(test)-len(test)%BATCH])

      test_datasets = D.TensorDataset(test,pos_tag_t,relation_t,position1_t,position2_t,labels_t)
      test_dataloader = D.DataLoader(test_datasets,BATCH,True,num_workers=2)
      #training
      for sentence,pos_tag,relation,pos1,pos2,tag in train_dataloader:#批训练
          #print("sentence:",sentence)
          #print("pos1:",pos1)
          #print("pos2:",pos2)

          sentence = Variable(sentence.cuda())
          pos_tag = Variable(pos_tag.cuda())
          relation = Variable(relation.cuda())
          pos1 = Variable(pos1.cuda())
          pos2 = Variable(pos2.cuda())
          tags = Variable(tag.cuda())
          """
          sentence = Variable(sentence)
          pos_tag = Variable(pos_tag)
          e1_head = Variable(relation)
          pos1 = Variable(pos1)
          pos2 = Variable(pos2)
          tags = Variable(tag)
          """
          y = model(sentence,pos_tag,relation,pos1,pos2)  
          #epoch_loss += loss.data.cpu().numpy()[0]
          loss = criterion(y, tags)
          total_loss += loss 
          #loss += l2_loss(params[1:]) * l2_rate
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()    
          y = np.argmax(y.data.cpu().numpy(),axis=1)
          #y = np.argmax(y.data.numpy(),axis=1)
          #print("train_y:",y)
          #print("train_tag",tag)
          for y1,y2 in zip(y,tag):
             # print("train_y1:",y1)
             # print("train_y2:",y2)
              if y1==y2:
                  acc+=1

              total+=1
      train_p.append(100*float(acc)/total)
      print ("train:",100*float(acc)/total,"%")
      print("train_loss:",(total_loss/total).data.cpu().numpy())
     # print((total_loss/len(train_dataloader.dataset)).cpu().data.numpy())
      #print((total_loss/len(train_dataloader.dataset)).data.numpy())
      #print(loss.cpu().data.numpy()) 

    
    
      #start testing
      acc_t=0
      total_t=0
      total_loss_t=0
      count_predict = [0,0,0,0,0,0,0,0,0,0]
      count_total = [0,0,0,0,0,0,0,0,0,0]
      count_right = [0,0,0,0,0,0,0,0,0,0]
      for sentence,pos_tag,relation,pos1,pos2,tag in test_dataloader:  

          sentence = Variable(sentence.cuda()) 
          pos_tag = Variable(pos_tag.cuda()) 
          relation = Variable(relation.cuda())
          pos1 = Variable(pos1.cuda())
          pos2 = Variable(pos2.cuda())
          tags = Variable(tag.cuda())
          """
          sentence = Variable(sentence)
          pos_tag = Variable(pos_tag)
          relation = Variable(relation)
          pos1 = Variable(pos1)
          pos2 = Variable(pos2)
          tag = Variable(tag)
           """
          y = model(sentence,pos_tag,relation,relation,pos1,pos2)
          loss = criterion(y, tags)
          total_loss_t += loss 

          y = np.argmax(y.data.cpu().numpy(),axis=1)
         # y = np.argmax(y.data.numpy(),axis=1)
         # print('testy:',y)
         # print('testtag:',tag)
          for y1,y2 in zip(y,tag):
             # print("test_y1:",y1)
              #print("test_y2:",y2)
              count_predict[y1]+=1
              count_total[y2]+=1
              if y1==y2:
                  count_right[y1]+=1
              total_t+=1

      print("test_loss:",(total_loss_t/total_t).data.cpu().numpy())
      precision = [0,0,0,0,0,0,0,0,0,0]
      recall = [0,0,0,0,0,0,0,0,0,0]
      for i in range(len(count_predict)):
          if count_predict[i]!=0 :
              precision[i] = float(count_right[i])/count_predict[i]

          if count_total[i]!=0:
              recall[i] = float(count_right[i])/count_total[i]    

      precision = sum(precision)/len(Relation2id)
      recall = sum(recall)/len(Relation2id)
      test_p.append(precision)
      test_r.append(recall)
      test_f.append((2*precision*recall)/(precision+recall))
      print ("准确率：",precision)
      print ("召回率：",recall)
      print ("f：", (2*precision*recall)/(precision+recall))
    #print("train_p:",count(train_p)/len(train_p))
    #print("test_p:",count(test_p)/len(test_p))
    #print("test_r:",count(test_r)/len(test_r))
    #print("test_f:",count(test_f)/len(test_f))
#start saveing
torch.save(model, "drive/论文/ChineseNRE-master更新后/data/model_BiLSTM_selfAtt_ch.pkl")
print ("model has been saved")

12
Baseline_Model(
  (word_emb): Embedding(39770, 300)
  (pos_tag_emb): Embedding(30, 30)
  (relation_emb): Embedding(82, 30)
  (pos1_emb): Embedding(165, 25)
  (pos2_emb): Embedding(165, 25)
  (rnn): LSTM(300, 200, batch_first=True, bidirectional=True)
  (attention_hidden): Linear(in_features=510, out_features=200, bias=False)
  (attention): Linear(in_features=200, out_features=1, bias=False)
  (classifier): Linear(in_features=400, out_features=12, bias=True)
)
epoch: 0


ValueError: ignored

In [0]:

s =[1,2,3,4]
print(count[s])

NameError: ignored

In [0]:
print(len(train_dataloader.dataset))